## MC simulations of fluids

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
from itertools import product

from ipywidgets import interact, interactive
import plotly.express as px
import plotly.graph_objects as go
from numba import jit, njit

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print('All good to go')
except:
    print('Okay we are not in Colab just proceed as if nothing happened')

## LJ model of a simple fluids, noble gases


**Check out reference values of [LJ fluid properties tabulated by NIST](https://www.nist.gov/mml/csd/chemical-informatics-research-group/lennard-jones-fluid-properties)**



### Initialize the system, and watch out for clashes!

- Density and unit cell number can be used to fully specify a lattice packed with LJ particles 

In [ ]:
def IC_pos(rho=0.88, N_cell=3):
    """
    Generate FCC lattice positions for Lennard-Jones particles.

    Parameters:
    rho (float): Number density (default 0.88).
    N_cell (int): Number of unit cells per dimension (default 3).

    Returns:
    pos (np.ndarray): Particle positions.
    L (float): Box length.
    N (int): Number of particles.
    """
    from itertools import product
    import numpy as np

    N = 4 * N_cell**3
    L = (N / rho)**(1/3)
    L_cell = L / N_cell

    base = np.array([[0, 0, 0],
                     [0, 0.5, 0.5],
                     [0.5, 0, 0.5],
                     [0.5, 0.5, 0]])

    pos = np.array([b + [i, j, k] for i, j, k in product(range(N_cell), repeat=3) for b in base])
    pos *= L_cell

    return pos, L, N


In [ ]:
import plotly.graph_objects as go

def cell_plotter(rho=0.88, N_cell=3, marker_size=5, marker_opacity=0.5):
    """
    Plot a 3D scatter of an FCC lattice for Lennard-Jones particles.

    Parameters:
    rho (float): Number density (default 0.88).
    N_cell (int): Number of unit cells per dimension (default 3).
    marker_size (float): Size of plotted particles (default 5).
    marker_opacity (float): Opacity of plotted particles (default 0.5).
    """
    pos, L, N = IC_pos(rho=rho, N_cell=N_cell)

    fig = go.Figure(data=[go.Scatter3d(
        x=pos[:, 0], y=pos[:, 1], z=pos[:, 2],
        mode='markers',
        marker=dict(size=marker_size, opacity=marker_opacity)
    )])

    fig.update_layout(
        title=f"FCC Lattice: ρ = {rho}, N = {N}",
        width=700, height=700,
        scene=dict(
            xaxis=dict(title='X', range=[0, L]),
            yaxis=dict(title='Y', range=[0, L]),
            zaxis=dict(title='Z', range=[0, L]),
            aspectmode='cube'
        )
    )

    fig.show()

In [ ]:
interactive(cell_plotter, rho=(0.5, 1.5, 0.1), N_cell=(1, 10, 1))

### [Periodic Boundary conditions and minimum image criterion](https://en.wikipedia.org/wiki/Periodic_boundary_conditions)

- If a particle leaves the box, it re-enters at the opposite side.
- What should be the distance between partiles? There is some mbiguity because distance between atoms may now include crossing the system boundary and re-entering. This is identical to introducing copies of the particles around the simulation box. We adopt **minimum image convention** by choosing the shortest distance possible

![](./pbc_fig1.gif)

![](./pbc_fig2.gif)

- By adopting minimum image convention we consider the closest of all 'image partners' (plus the original) of every atom for calculating interaction. The animation below highlights the image partners of the two red atoms that are closest to the green atom. 

- To minimize distance between points we evaluate distances along x, y and z dimension and choose the smallest possible one. 


In [ ]:
def pbc_basic(x, L):
  '''Wrap a single components into [-L/2, L/2) assuming box centered at origin'''
    
  if   x >=  L/2: x -= L  

  elif x <= -L/2: x += L

  return x


@njit
def pbc_wrap(r_vec, L):
    """
    Wrap all vector components into [-L/2, L/2) assuming box centered at origin.
    """
    return (r_vec + L/2) % L - L/2

### Computing pairwise distances and energies

In [ ]:
@njit
def getE_tot(pos, L, trunc, sig=1.0, eps=1.0):
    """
    Compute total Lennard-Jones energy with cutoff and PBC.
    """
    N = len(pos)
    energy = 0.0
    trunc_sq = trunc**2

    for i in range(N - 1):
        for j in range(i + 1, N):
            r_vec = pbc_wrap(pos[i] - pos[j], L)
            r_sq = np.sum(r_vec**2)

            if r_sq < 1e-12:
                continue  # avoid singularity
                
            if r_sq <= trunc_sq:
                inv_r_sq = sig**2 / r_sq
                energy += 4 * eps * (inv_r_sq**3 - inv_r_sq**1.5)

    return energy

@njit
def E_disp(pos, L, trunc, j, r_j, sig=1.0, eps=1.0):
    """
    Compute interaction energy of particle j at trial position r_j.
    """
    N = len(pos)
    energy = 0.0
    trunc_sq = trunc**2

    for i in range(N):
        if i == j:
            continue
        r_vec = pbc_wrap(pos[i] - r_j, L)
        r_sq = np.sum(r_vec**2)

        if r_sq < 1e-12:
            continue
        if r_sq <= trunc_sq:
            inv_r_sq = sig**2 / r_sq
            energy += 4 * eps * (inv_r_sq**3 - inv_r_sq**1.5)

    return energy

### MC engine for LJ fluid in 3D (NVT ensemble)
Now that the main helper functions are set up we can put together a main Monte Carlo engine that loops through randomly selected particles and attemps their displacement via Metropolis Criterion. 

In [ ]:
import numpy as np
from numba import njit

@njit
def run_MC_LJ(pos, L, T, trunc=4.0, disp=0.5, steps=10000, freq=100):

    N = len(pos)
    n_snapshots = steps // freq + 1
    confs = np.empty((n_snapshots, N, 3))
    es = np.empty(n_snapshots)

    # Initial energy
    E_tot, _ = getE_tot(pos, L, trunc)
    conf_index = 0
    confs[conf_index] = pos
    es[conf_index] = E_tot
    conf_index += 1

    for step in range(steps):
        
        j = np.random.randint(N)

        # Generate displacement
        dx = np.random.uniform(-disp, disp)
        dy = np.random.uniform(-disp, disp)
        dz = np.random.uniform(-disp, disp)
        delta = np.array([dx, dy, dz])

        r_j_new = pbc_wrap(pos[j] + delta, L)

        # Energy change for displacement
        e_old = E_disp(pos, L, trunc, j, pos[j])
        e_new = E_disp(pos, L, trunc, j, r_j_new)
        dE = e_new - e_old


        if dE <= 0.0 or np.random.rand() < np.exp(-dE / T):

            pos[j] = r_j_new
            E_tot += dE

        if step % freq == 0:

            confs[conf_index] = pos
            es[conf_index] = E_tot
            conf_index += 1

    return confs, es


### Running MCMC simulation on LJ system

In [ ]:
rho=0.88
pos, L, N = IC_pos(rho, N_cell=3)

params = dict(L = L,
              T = 1, 
              steps=1000000, 
              trunc=4, 
              disp=0.5,
              freq=100)

In [ ]:
#%time 
confs, es = run_MC_LJ(pos, **params)

In [ ]:
plt.plot(es) 

In [ ]:
n = params['steps']//params['freq']

@interact(i=(0, n-1))
def viz_sim_lj(i=0):

  fig = go.Figure(data=[go.Scatter3d(
        x=pos[i,0],
        y=pos[i,1],
        z=pos[i,2],
        mode='markers',
        marker=dict(
            size=5,  # Adjust size of markers
            opacity=0.5  # Adjust opacity of markers
        )
    )])

  return fig

In [ ]:
def compute_g_r(configs, L, dr=0.05, r_max=None):
    """
    Compute radial distribution function g(r) from a list of configurations.

    Parameters:
    configs (list): List of N x 3 arrays of particle positions.
    L (float): Box length.
    dr (float): Bin width for r.
    r_max (float): Max distance to consider (default L/2).

    Returns:
    r (np.ndarray): Array of distance bin centers.
    g_r (np.ndarray): Radial distribution function values.
    """
    import numpy as np

    if r_max is None:
        r_max = L / 2

    n_bins = int(r_max / dr)
    hist = np.zeros(n_bins)
    norm = 0

    for pos in configs:
        N = len(pos)
        for i in range(N - 1):
            for j in range(i + 1, N):
                r_vec = pos[i] - pos[j]
                # Apply minimum image convention
                r_vec -= L * np.round(r_vec / L)
                r = np.linalg.norm(r_vec)
                if r < r_max:
                    bin_index = int(r / dr)
                    hist[bin_index] += 2  # each pair counts once, but both particles contribute

        norm += N * (N - 1)

    # Normalize g(r)
    rho = len(configs[0]) / L**3
    r = np.linspace(0, r_max, n_bins, endpoint=False) + dr / 2
    shell_vol = 4/3 * np.pi * ((r + dr/2)**3 - (r - dr/2)**3)
    ideal_counts = norm * rho * shell_vol / len(configs)
    g_r = hist / ideal_counts

    return r, g_r


In [ ]:
r, g_r = compute_g_r(confs_prod, L, dr=0.05)

### Problems

**1. Implementing PBC and minimal image convention methods outlined on  [Wikipeida](https://en.wikipedia.org/wiki/Periodic_boundary_conditions) in python.**

- Write a few functions that take positions of N particles in 3D with shape (N,3) return new coordinates and all inter-particle distances. 
> You can generate random positions using ```np.random``` make sure you have enough particles outside of box to test your functions. 
> Try implementing your functions using numpy methods instead of having multiple for loops. That way your functions will run singnificantly faster.


```python
def pbc_dists(pos, L):

    ...
    
    return pos, dists

```

 - Evaluate distribution of energies of your random positions before and after applying pbc_dists.
 
 
**2. Simulating 2D random walk with PBCs.**

 - Create a sequence of images or better yet an animation showing the temporal evolution of 10 independent random walkers in 100 by 100 square. Consult the random walk section and notebooks to refresh your memory of random walk simulations.
 - Calculate the root mean square displacement from the origin and show how it scales with time (steps)!
  
 
**2. Run MC simulations of LJ fluid at several temperatures to identify critical temperature.**

 - At each temperature evaluate heat capacity and RDFs.
 - Plot how energy, heat capacity and RDF change as a function of temperature.
 - Study dependence on sampling efficiency on magnitude of particle displacement.